In [11]:
from fastai.basics import *
from fastai.distributed import *
from fastai.learner import *
from fastai.vision import *
from fastai.vision.data import *
from fastai.data import *
from fastai.vision.all import *
import torch.distributed as dist
from torch._utils import _flatten_dense_tensors, _unflatten_dense_tensors
from torch.utils.data import TensorDataset

In [20]:
FP16_DeepSpeedZeroOptimizer_Stage1

In [17]:
def Zero_Stage1(params, lr, mom=0.9, sqr_mom=0.99, eps=1e-5, wd=0.01, decouple_wd=True):
    "A `Optimizer` for Adam with `lr`, `mom`, `sqr_mom`, `eps` and `params`"
    cbs = [weight_decay] if decouple_wd else [l2_reg]
    cbs += [partial(average_grad, dampening=True), average_sqr_grad, step_stat, adam_step]
    return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)

In [16]:
n_batches=20
bs=64
seed=1337#random.randint(0,2**32-1)

def synth_dbunch(a=2, b=3, bs=16, n_train=10, n_valid=2, cuda=False):
    def get_data(n):
        x = torch.randn(bs*n, 1)
        return TensorDataset(x, a*x + b + 0.1*torch.randn(bs*n, 1))
    train_ds = get_data(n_train)
    valid_ds = get_data(n_valid)
    device = rank_distrib() #default_device() if cuda else None
    train_dl = TfmdDL(train_ds, bs=bs, shuffle=True, num_workers=1)
    valid_dl = TfmdDL(valid_ds, bs=bs, num_workers=1)
    return DataLoaders(train_dl, valid_dl, device=device)
    
    
def synth_learner(n_trn=10, n_val=2, cuda=False, lr=1e-3,opt_func=partial(SGD, mom=0.9), data=None, model=None, **kwargs):
    if data is None: data=synth_dbunch(n_train=n_trn,n_valid=n_val, cuda=cuda)
    if model is None: model=RegModel()
    return Learner(data, model, lr=lr, loss_func=MSELossFlat(),
                   opt_func=opt_func, **kwargs)

with no_random(seed): 
    model=nn.Sequential(nn.Linear(1,1))
    db=synth_dbunch(bs=bs,n_train=n_batches,n_valid=n_batches,cuda=True)
    learn = synth_learner(model=model,data=db)
    learn.model.to(learn.dls.device)
    learn.to_fp16()
    with learn.distrib_ctx(): learn.fit(10, lr=0.01)

epoch,train_loss,valid_loss,time
0,7.596642,0.429888,00:00
1,3.479436,0.016870,00:00
2,1.867872,0.027477,00:00
3,1.099680,0.014495,00:00
4,0.682311,0.010631,00:00
5,0.437054,0.009986,00:00
6,0.286092,0.009920,00:00
7,0.190431,0.009939,00:00
8,0.128757,0.009962,00:00
9,0.088462,0.009928,00:00


In [22]:
list(learn.model.parameters())

[Parameter containing:
 tensor([[2.0038]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([3.0038], device='cuda:0', requires_grad=True)]

Starting here

In [41]:
parameter_parallel_size = None

In [42]:
num_distrib()

0

In [49]:
data_parallel_size = 2 #num_distrib()

In [50]:
parameter_parallel_size = parameter_parallel_size or data_parallel_size

In [51]:
self.opt=opt

NameError: name 'opt' is not defined

In [52]:
all_gather_partitions=True

In [53]:
all_gather_size=500000000

In [54]:
flat_params=torch.flatten(torch.tensor([[1,2],[3,4]]))

In [55]:
parameter_parallel_size

2

In [90]:
learn.opt.param_lists

(#1) [[Parameter containing:
tensor([[2.0038]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([3.0038], device='cuda:0', requires_grad=True)]]

In [67]:
rank=rank_distrib()

In [61]:
flat_params

tensor([1, 2, 3, 4])

In [76]:
params=range(rank*flat_params.numel()//parameter_parallel_size,(rank+1)*flat_params.numel()//parameter_parallel_size)
params=flat_params[params]

In [86]:
rank_distrib(),num_distrib()

(0, 0)

In [83]:
learn.opt.all_params(with_grad=True)[0][0]

Parameter containing:
tensor([[2.0038]], device='cuda:0', requires_grad=True)

In [ ]:
class ZeroStage1(Optimizer)
    def __init__(self,params, cbs, train_bn=True, **defaults):
        super().__init__(params, cbs, train_bn=True, **defaults)
        par
    def step(self): 
        for p,pg,state,hyper in self.all_params(n=slice(None, None, None),with_grad=True):
            for cb in self.cbs: state = _update(state, cb(p, **{**state, **hyper}))
            self.state[p] = state
    def partition(self):
        

In [ ]:
def opt_func(params,lr,opt,**kwargs):
    opt=opt(opt)
    

In [94]:
Optimizer??

In [93]:
learn.opt.all_params??

In [97]:
learn.to_non_native_fp16??

In [96]:
NonNativeMixedPrecision??

In [98]:
ModelToHalf??